#<font color="yellow">This sample shows how we could leverage an OCR software to digitalize the payment worflow </font>
In this example we are using a pdf reader to parse an invoice, extract relevant fields and create a payment
Once the payment is created we then attach the invoice to it leveraging the attachement API

In real life a commercial OCR/AI tool may be used for better accuracy and ability to handle different type of lay out/invoices, as well as a validation workflow in the Kyriba Application

## <font color='yellow'>Import 3d party library</font>

In [ ]:
#@title Install library for OCR (here a pdf reader)
!pip install pdfplumber -q
!pip install tabula-py

## <font color='yellow'>Code for token generation and API management</font>

In [ ]:
#@title Select **servicePack** the Sample should run
from subprocess import getstatusoutput
servicePack = "DEMO" 

if servicePack == 'DEMO':
  branch = 'main'
else:
  branch = servicePack
print(branch)
rm = getstatusoutput("rm Colab_shared_code_for_API_usage -rf ") 
clone = getstatusoutput("git clone -l -s --branch " +  branch + " https://github.com/kyriba/Colab_shared_code_for_API_usage.git") 
import importlib
try:  
  importlib.reload(sample_requests)
except:
  print()
from Colab_shared_code_for_API_usage import sample_requests

In [3]:
#@title Import custom modules
import json as js
import base64
import pdfplumber
from datetime import datetime,date

## <font color='yellow'>Connect and  retrieve token</font>

In [ ]:
#@title Connect and get token
token = sample_requests.login()


## <font color='yellow'>Process Payment File</font>

In [ ]:
#@title Upload invoice in PDF format

from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  invoice = fn

In [ ]:
#@title Parse and extract data

pdf_file = open(invoice, 'rb')
read_pdf = pdfplumber.open(pdf_file)
number_of_pages = len(read_pdf.pages)
page = read_pdf.pages[0]
page_content = page.extract_text()

def get_data(page_content):
    _dict = {}
    page_content_list = page_content.splitlines()
    for line in page_content_list:
        if ':' not in line:
            continue
        key, value = line.split(':')
        _dict[key.strip()] = value.strip()
    return _dict


page_data = get_data(page_content)
str_data = js.dumps(page_data, indent=4)
json_data= js.loads(str_data)
print(json_data)

In [ ]:
#@title Read data table
from tabula import read_pdf

#reads table from pdf file
df = read_pdf(invoice,pages="all") 

df_out = df[0].to_json(orient='records')[1:-1].replace('},{', '} {')
df_data = js.loads(df_out)
print(df_data)

In [ ]:
#@title Import the Payment in Kyriba
from datetime import datetime,date
third_party_pype = "Regular" #@param ["Regular", "One-Time"] {type:"string"}
traces = False #@param {type:"boolean"}

today = date.today()
now = datetime.now()
date= json_data["Date"]
description = df_data["Description"]
amount = df_data["Amount"].split()[0]
currency = df_data["Amount"].split()[1]
invoice_num = json_data["Invoice N°"]
activity = df_data["Activity"]
debtor = json_data["Company Name"]
cdtr_acct = json_data["IBAN"].replace(' ', '')  
cdtr_bic = json_data["BIC"].replace(' ', '')
cdtr_name = json_data["Account Name"]
cdtr_country = json_data["IBAN"][:2]
cdtr_address = json_data["Company address"]

dt_string = str(today) + 'T' + now.strftime("%H:%M:%S")
reference = 'RTP' + dt_string

if (third_party_pype == "Regular") :
  template = 'DEMO_PY_V2'
  data = 'T;' + debtor + ';' + date +';'+ amount + ';' + currency + ';' + cdtr_acct + ';' + reference + \
  '\nINV;' + invoice_num +';' + date + ';' + amount + ';' + activity + ';' + description
elif (third_party_pype == "One-Time")  :
  template = 'DEMO_PY_V3';
  data = 'T;' + debtor + ';' + date +';'+ amount + ';' + currency + ';' + cdtr_name + ';'\
 + cdtr_country + ';' + cdtr_address + ';' + cdtr_acct + ';' + cdtr_bic + ';' + reference + \
'\nINV;' + invoice_num +';' + date + ';' + amount + ';' + activity + ';' + description

filename = dt_string + '-test.txt'

#Import payment
response=sample_requests.import_data(token,data,filename,template,True,traces)

get_status = sample_requests.get_results(token, '/v1/payment/transfers/status?filter=reference==' + reference)
status = js.loads(get_status)["results"][0]["status"]
print()
print("Payment with reference " + reference + " created and with status " + status) 

## <font color='yellow'>Manage attachment</font>

In [ ]:
#@title Upload the invoice in Kyriba

pdf_file = open(invoice, 'rb')
# Transform to binary
binary_file_data = pdf_file.read()

payload={}
files=[
  ('file0', (invoice, binary_file_data, 'multipart/form-data'))
]

result = sample_requests.post_results(token,"/v1/data/files", payload, files)
fileId = result[0]['fileId']

print("Invoice " + invoice + " uploaded into Kyriba")

In [ ]:
#@title Attach the invoice to the Payment
payload = '[" '+ fileId +' "]'

post_response = sample_requests.post_results(token,"/v1/attachments/attach/TRANSFER?filter=reference==" + reference,payload,{},headers= {'content-type':'application/json'})

print(invoice + " attached to payment with reference " + reference)